In [2]:
#!pip install --upgrade virtualenv



'pip3' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!pip install opencv-python

!pip install opencv-contrib-python

!pip install Pillow

  Using cached https://files.pythonhosted.org/packages/85/17/bad54f67bbe27d88ba520c3f59315e95b4e254cd28767c20accacb0597d8/opencv_python-4.2.0.34-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/c6/a1/d0fd1ea9d5417437e6271e7f88d3ed155f64a979d3ccd926a3b21d4404dd/opencv_contrib_python-4.2.0.34-cp37-cp37m-win_amd64.whl


In [4]:
import numpy as np

In [9]:
import sqlite3
conn = sqlite3.connect('database.db')
c = conn.cursor()
sql = """
DROP TABLE IF EXISTS users;
CREATE TABLE users (
           id integer unique primary key autoincrement,
           name text,
           emotion text,
           timestamp datetime
);
"""
c.executescript(sql)
conn.commit()
conn.close()


In [12]:
import cv2
import numpy as np 
import sqlite3
import os
conn = sqlite3.connect('database.db')
if not os.path.exists('./dataset'):
    os.makedirs('./dataset')
c = conn.cursor()
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)
uname = input("Enter your name: ")
c.execute('INSERT INTO users (name,emotion) VALUES (?,?)', (uname,emot))
#c.execute('INSERT INTO users () VALUES (?)', (emot,))
uid = c.lastrowid
sampleNum = 0
while True:
  ret, img = cap.read()
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  faces = face_cascade.detectMultiScale(gray, 1.3, 5)
  for (x,y,w,h) in faces:
    sampleNum = sampleNum+1
    cv2.imwrite("dataset/User."+str(uid)+"."+str(sampleNum)+".jpg",gray[y:y+h,x:x+w])
    cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), 2)
    roi_gray = gray[y:y + h, x:x + w]
    cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
    cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
    #cv2.rectangle(gray, (x, y), (x + w, y + h), (0, 255, 0), 1)
    #predicting the emotion
    yhat= loaded_model.predict(cropped_img)
    cv2.putText(img, labels[int(np.argmax(yhat))], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
    print("Emotion: "+labels[int(np.argmax(yhat))])
    cv2.waitKey(100)
  cv2.imshow('img',img)
  cv2.waitKey(1);
  if sampleNum > 20:
    break
emot = labels[int(np.argmax(yhat))]

cap.release()
conn.commit()
conn.close()
cv2.destroyAllWindows()

Enter your name: karan
Emotion: Surprise
Emotion: Surprise
Emotion: Surprise
Emotion: Fear
Emotion: Sad
Emotion: Fear
Emotion: Fear
Emotion: Fear
Emotion: Fear
Emotion: Fear
Emotion: Fear
Emotion: Sad
Emotion: Fear
Emotion: Sad
Emotion: Surprise
Emotion: Fear
Emotion: Fear
Emotion: Fear
Emotion: Fear
Emotion: Fear
Emotion: Fear


In [38]:
import os
import cv2
import numpy as np 
from PIL import Image
recognizer = cv2.face.LBPHFaceRecognizer_create()
path = 'dataset'
if not os.path.exists('./recognizer'):
    os.makedirs('./recognizer')
def getImagesWithID(path):
  imagePaths = [os.path.join(path,f) for f in os.listdir(path)]
  faces = []
  IDs = []
  for imagePath in imagePaths:
    faceImg = Image.open(imagePath).convert('L')
    faceNp = np.array(faceImg,'uint8')
    ID = int(os.path.split(imagePath)[-1].split('.')[1])
    faces.append(faceNp)
    IDs.append(ID)
    cv2.imshow("training",faceNp)
    cv2.waitKey(10)
  return np.array(IDs), faces
Ids, faces = getImagesWithID(path)
recognizer.train(faces,Ids)
recognizer.save('recognizer/trainingData.yml')
cv2.destroyAllWindows()


In [13]:
import cv2
import numpy as np 
import sqlite3
import os
conn = sqlite3.connect('database.db')
c = conn.cursor()
fname = "recognizer/trainingData.yml"
if not os.path.isfile(fname):
  print("Please train the data first")
  exit(0)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read(fname)
while True:
  ret, img = cap.read()
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  faces = face_cascade.detectMultiScale(gray, 1.3, 5)
  for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
    ids,conf = recognizer.predict(gray[y:y+h,x:x+w])
    c.execute("select name from users where id = (?);", (ids,))
    result = c.fetchall()
    name = result[0][0]
    if conf < 50:
      cv2.putText(img, name, (x+2,y+h-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (150,255,0),2)
    else:
      cv2.putText(img, 'No Match', (x+2,y+h-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),2)
    roi_gray = gray[y:y + h, x:x + w]
    cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
    cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
    #cv2.rectangle(gray, (x, y), (x + w, y + h), (0, 255, 0), 1)
    #predicting the emotion
    yhat= loaded_model.predict(cropped_img)
    cv2.putText(img, labels[int(np.argmax(yhat))], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
    print("Emotion: "+labels[int(np.argmax(yhat))])
  
  cv2.imshow('Face Recognizer',img)
  k = cv2.waitKey(30) & 0xff
  if k == 27:
    break
cap.release()
cv2.destroyAllWindows()

Emotion: Surprise
Emotion: Surprise
Emotion: Surprise
Emotion: Surprise
Emotion: Surprise
Emotion: Surprise
Emotion: Sad
Emotion: Fear
Emotion: Surprise
Emotion: Fear
Emotion: Surprise
Emotion: Sad
Emotion: Sad
Emotion: Surprise
Emotion: Surprise
Emotion: Surprise
Emotion: Sad
Emotion: Fear
Emotion: Sad
Emotion: Surprise
Emotion: Surprise
Emotion: Neutral
Emotion: Surprise
Emotion: Fear
Emotion: Sad
Emotion: Sad
Emotion: Fear
Emotion: Fear
Emotion: Sad
Emotion: Sad
Emotion: Sad
Emotion: Neutral
Emotion: Surprise
Emotion: Fear
Emotion: Sad
Emotion: Sad
Emotion: Sad
Emotion: Sad
Emotion: Sad
Emotion: Sad
Emotion: Sad
Emotion: Sad
Emotion: Sad
Emotion: Angry
Emotion: Sad
Emotion: Fear
Emotion: Sad
Emotion: Fear
Emotion: Fear
Emotion: Neutral
Emotion: Happy
Emotion: Neutral
Emotion: Neutral
Emotion: Happy
Emotion: Neutral
Emotion: Happy
Emotion: Happy
Emotion: Happy
Emotion: Happy
Emotion: Happy
Emotion: Neutral
Emotion: Neutral
Emotion: Happy
Emotion: Fear
Emotion: Happy
Emotion: Happy
Em

In [24]:
!git clone https://github.com/atulapra/Emotion-detection.git


'git' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
# load json and create model
from __future__ import division
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy
import os
import numpy as np
import cv2

#loading the model
json_file = open('fer.json', 'r')
loaded_model_json = json_file.read()

json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("fer.h5")
print("Loaded model from disk")

#setting image resizing parameters
WIDTH = 48
HEIGHT = 48
x=None
y=None
labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

#loading image
full_size_image = cv2.imread("smile5.jpg")
print("Image Loaded")
gray=cv2.cvtColor(full_size_image,cv2.COLOR_BGR2GRAY)
face = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
faces = face.detectMultiScale(gray, 1.51 , 5)

#detecting faces
for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
        cv2.rectangle(full_size_image, (x, y), (x + w, y + h), (0, 255, 0), 1)
        #predicting the emotion
        yhat= loaded_model.predict(cropped_img)
        cv2.putText(full_size_image, labels[int(np.argmax(yhat))], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
        print("Emotion: "+labels[int(np.argmax(yhat))])
cv2.imshow('Emotion', full_size_image)
cv2.waitKey()


Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Loaded model from disk
Image Loaded


-1

In [2]:
cv2.imshow('Emotion', full_size_image)

In [21]:
full_size_image = cv2.imread("smile5.jpg")
print("Image Loaded")
gray=cv2.cvtColor(full_size_image,cv2.COLOR_RGB2GRAY)
face = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
faces = face.detectMultiScale(gray, 1.3  , 10)

#detecting faces
for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
        cv2.rectangle(full_size_image, (x, y), (x + w, y + h), (0, 255, 0), 1)
        #predicting the emotion
        yhat= loaded_model.predict(cropped_img)
        cv2.putText(full_size_image, labels[int(np.argmax(yhat))], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
        print("Emotion: "+labels[int(np.argmax(yhat))])
cv2.imshow('Emotion', full_size_image)
cv2.waitKey()

    

Image Loaded
Emotion: Happy


-1

In [22]:
labels[int(np.argmax(yhat))]

'Happy'

In [32]:
#full_size_image = cv2.imread("smile6.jpg")
#print("Image Loaded")
#gray=cv2.cvtColor(full_size_image,cv2.COLOR_RGB2GRAY)
#clr=cv2.cvtColor(full_size_image,cv2.COLOR_GRAY2BGR)
face = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
faces = face.detectMultiScale(gray, 1.51  , 5)
#cv2.imshow('org',clr)
cv2.imshow('gray',gray)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [33]:
for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
        cv2.rectangle(gray, (x, y), (x + w, y + h), (0, 255, 0), 1)
        #predicting the emotion
        yhat= loaded_model.predict(cropped_img)
        cv2.putText(full_size_image, labels[int(np.argmax(yhat))], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
        print("Emotion: "+labels[int(np.argmax(yhat))])
cv2.imshow('Emotion', gray)
cv2.waitKey()


Emotion: Happy


-1

In [4]:
yhat

array([[5.1903939e-01, 6.0865746e-05, 3.0363172e-01, 1.8868479e-03,
        3.1362856e-03, 1.6464268e-01, 7.6021412e-03]], dtype=float32)